# Heart-masked telegram wordcloud

In [ ]:
import string
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd
import json
with open("result.json", 'r', encoding="utf-8") as file:
    json_data = file.read()

parsed_json = json.loads(json_data)
df = pd.DataFrame(parsed_json["messages"])
message_texts = df['text']
# Мы скипаем все сложные сообщения, оставляем только те, что состоят только из строк. В количественном отношении у меня сложны сообщений было меньше 0.1%
message_texts = message_texts.loc[(message_texts.apply(lambda x: type(x) == str))]
# А тут скипаем все пустые ячейки
message_texts.loc[message_texts != ""]

In [ ]:
# Поработаем со стоп словами
from nltk.corpus import stopwords
stop_words = set(stopwords.words('russian'))
with open("ru-stop_words.txt", 'r', encoding='utf-8') as f:
    words = [line.rstrip("\n") for line in f.readlines()]
    stop_words.update(words)
stop_words.update("просто это тебе знаю хочу буду щас почему сегодня ладно понял чо думаю завтра ".split())

In [ ]:
from nltk.tokenize import word_tokenize
def preprocess_text(message_text):
    tokens = word_tokenize(message_text)
    # Тут мы чистим стоп-слова и пунктуационные знаки
    message_text = " ".join([word.lower() for word in tokens if not (word in stop_words or word in string.punctuation)])
    return message_text

In [ ]:
filtered_texts = message_texts.apply(preprocess_text)
filtered_texts

In [ ]:
from PIL import Image, ImageOps
import numpy as np
mask = Image.open('img/mask.png')
# так как изначально моя маска была кривая и косая, мне пришлось немного повыёживаться и сделать из неё контрастную
# color1 = (255, 255, 255)
# color2 = (0, 0, 0)
# mask = ImageOps.colorize(original_image.convert('L'), color2, color1)
# mask_array = np.array(mask, dtype=float) / 255.0
# mask_array = (1 - np.where(mask_array < 1.0, 0, mask_array)) * 255
all_words = " ".join(filtered_texts)
plt.imshow(mask)

In [ ]:
# word_counts = Counter(all_words)
# top_words = dict(word_counts.most_common(100))
wordcloud = WordCloud(width = 1000, height = 1000, random_state=1,
                      background_color='black', colormap='Set1',
                      collocations=False, stopwords=stop_words, mask=np.array(mask))
# wordcloud.generate_from_frequencies(top_words)
wordcloud.generate(all_words)
plt.figure(figsize=(5,5), dpi=300)
plt.axis('off')
plt.imshow(wordcloud)
plt.tight_layout()
plt.savefig("img/result.png", bbox_inches='tight')